In [249]:
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models.phrases import Phraser, Phrases
import pandas as pd
import re

In [8]:
data = pd.read_csv('court-V-N.csv', header=None)
data.head()

,0,1,2,3,4
0,СУД,АРЕСТОВАТЬ,ПОДПОЛКОВНИК,ЗАМНАЧАЛЬНИК,ОТДЕЛ
1,СУД,ВЕСТИ,ПЕРЕГОВОРЫ,ПОСТАВКА,БУДУЩИЙ
2,СУД,ВЗЯТЬ,РЕАЛИЗАЦИЯ,ЗАКОН,ОБЯЗАТЕЛЬНЫЙ
3,СУД,ВЫКУПИТЬ,ГОСУДАРСТВО,АКЦИЯ,РАНЕЕ
4,СУД,ГОВОРИТЬ,ПРЕСС-СЕКРЕТАРЬ,ЕВГЕНИЙ,ВРЕМЕНИТЬ


In [29]:
rows = list(map(list, data.values))
tokens = [j for i in rows for j in i]

In [123]:
print(len(rows), 'предложений', ';', len(tokens), 'слов')

1020 предложений ; 5100 слов


Выделим наиболее типичные для суда действия, применив 2 различных метода (оба являются вариациями алгоритма nltk bigrams, выбирают только те коллокации, которые встречаются больше 10 раз, но используют различные метрики):

#### NLTK Bigrams with PMI and Likelihood metrics

In [132]:
def freq_bigrams(tokens, metrics, nbest='all', freq_filter=0):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    metrics_={'pmi':bigram_measures.pmi, 'lr':bigram_measures.likelihood_ratio}
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(freq_filter)
    if nbest == 'all':
        return finder.score_ngrams(metrics_[metrics])
    nbest = finder.nbest(metrics_[metrics], nbest)
    return nbest

In [242]:
bigrams_pmi = freq_bigrams(tokens, 'pmi', freq_filter=10)
bigrams_pmi = [''.join(i[0]).lower().strip() for i in bigrams_pmi]
bigrams_pmi[:10]

['выдать санкция',
 'наложить арест',
 'санкция арест',
 'удовлетворить ходатайство',
 'признать виновная',
 'принять решение',
 'удовлетворить иск',
 'вынести решение',
 'суд рассмотреть',
 'суд дать']

In [243]:
bigrams_lr = freq_bigrams(tokens, 'lr', freq_filter=10)
bigrams_lr = [''.join(i[0]).lower().strip() for i in bigrams_lr]
bigrams_lr[:10]

['суд суд',
 'суд признать',
 'выдать санкция',
 'суд удовлетворить',
 'удовлетворить иск',
 'удовлетворить ходатайство',
 'принять решение',
 'санкция арест',
 'наложить арест',
 'суд вынести']

In [108]:
import csv
for i, l in enumerate([bigrams_pmi, bigrams_lr]):
    with open("bigrams_{}.csv".format(['pmi', 'lr'][i]),'w') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        wr.writerows(l)

Пользуясь словарями, интуицией и т.п. составить золотой стандарт – ранжированный топ 10 коллокаций.

In [139]:
golden_standard = pd.read_csv('gold_standard.csv', header=None)

Сравним топ-10 коллокаций в каждом из списков:

In [140]:
results = pd.concat([pd.Series(bigrams_pmi[:10]), pd.Series(bigrams_lr[:10]), golden_standard], axis=1)
results.columns = ['pmi', 'lr', 'golden']
results

,pmi,lr,golden
0,выдать санкция,суд суд,признать виновная
1,наложить арест,суд признать,вынести приговор
2,санкция арест,выдать санкция,взять перерыв
3,удовлетворить ходатайство,суд удовлетворить,выслушать защита
4,признать виновная,удовлетворить иск,вызвать свидетель
5,принять решение,удовлетворить ходатайство,принять решение
6,удовлетворить иск,принять решение,назначить наказание
7,вынести решение,санкция арест,вести переговоры
8,суд рассмотреть,наложить арест,освободить залог
9,суд дать,суд вынести,удовлетворить ходатайство


In [273]:
bigrams_pmi_all = freq_bigrams(tokens, 'pmi', freq_filter=0)
for i in range(len(bigrams_pmi_all)):
    bigrams_pmi_all[i] = ((bigrams_pmi_all[i][0][0].lower().strip(), bigrams_pmi_all[i][0][1].lower().strip()), 
                          bigrams_pmi_all[i][1])
bigrams_pmi_all_ranks = {bi[0]:i+1 for i, bi in enumerate(bigrams_pmi_all)}

In [275]:
bigrams_lr_all = freq_bigrams(tokens, 'lr', freq_filter=0)
for i in range(len(bigrams_lr_all)):
    bigrams_lr_all[i] = ((bigrams_lr_all[i][0][0].lower().strip(), bigrams_lr_all[i][0][1].lower().strip()), 
                          bigrams_lr_all[i][1])
bigrams_lr_all_ranks = {bi[0]:i+1 for i, bi in enumerate(bigrams_lr_all)}

In [253]:
golden_dict = [(i.split()[0], i.split()[1]) for i in results['golden']]

In [277]:
ranks_golden = range(1, 11)
ranks_lr = [bigrams_lr_all_ranks[i] for i in golden_dict]
ranks_pmi = [bigrams_pmi_all_ranks[i] for i in golden_dict]
print('gold', ranks_golden)
print('lr', ranks_lr)
print('pmi', ranks_pmi)

gold range(1, 11)
lr [14, 88, 118, 1592, 1479, 7, 2029, 305, 13, 6]
pmi [1967, 1982, 636, 1250, 1043, 1980, 1796, 5, 926, 1928]


In [281]:
print('Spearman(lr, prmi)')
print(spearmanr(ranks_lr, ranks_pmi), '\n')
print('Spearman(lr, golden)')
print(spearmanr(ranks_lr,ranks_golden), '\n')
print('Spearman(pmi, golden)')
print(spearmanr(ranks_pmi, ranks_golden))

Spearman(lr, prmi)
SpearmanrResult(correlation=-0.3696969696969697, pvalue=0.2930500752928223) 

Spearman(lr, golden)
SpearmanrResult(correlation=-0.23636363636363633, pvalue=0.5108853175152002) 

Spearman(pmi, golden)
SpearmanrResult(correlation=-0.33333333333333326, pvalue=0.34659350708733405)


Во всех трех попарных корреляциях наблюдается невысокая обратная зависимость ранга, о чем говорит небольшие мо модулю 
отрицательные коэффициенты. Чем выше ранг в списке коллокаций из алгоритма nltk, тем ниже ранг в ЗС (хотя на 10 объектах говорить о каких-то корреляциях, конечно, нет особо смысла...).